In [34]:
AGENT_NAME = '20 bin PPO 500 results\default_PPO_citylearn_challenge_2022_phase_2_Building_6_20_bins_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = r'20 bin PPO 500 results\furthest action results' + '/'
EPS = 0.10
ATK_NAME = f'furthest action {EPS}'

In [35]:
from stable_baselines3 import PPO

from citylearn.data import DataSet

from art.attacks.evasion import AutoConjugateGradient as ACG
from art.utils import to_categorical

import pandas as pd
import numpy as np

import KBMproject.utilities as utils

import json
from tqdm import tqdm

%matplotlib inline

In [36]:
schema = DataSet.get_schema(DATASET_NAME)

In [37]:
try: #try to load CityLearn schema
    schema = DataSet.get_schema(DATASET_NAME)
except: #load saved schema otherwise
    with open(DATASET_NAME, 'r') as file:
        schema = json.load(file)

Define RL agent

In [38]:
agent = PPO.load(path=f"{AGENT_NAME}",
                 print_system_info=True)
print('Model loaded from storage')

== CURRENT SYSTEM INFO ==
- OS: Windows-10-10.0.22631-SP0 10.0.22631
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.1
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

== SAVED MODEL SYSTEM INFO ==
- OS: Windows-10-10.0.19045-SP0 10.0.19045
- Python: 3.10.12
- Stable-Baselines3: 1.8.0
- PyTorch: 1.12.0
- GPU Enabled: True
- Numpy: 1.25.1
- Gym: 0.21.0

Model loaded from storage


In [39]:
bins = agent.action_space[0].n
env = utils.make_discrete_env(schema=schema,  
                        action_bins=bins,
                        seed=42)

In [40]:
cols = env.observation_names

In [41]:
observation_masks = np.ones(agent.observation_space.shape)
observation_masks[0:6] = 0 #mask time features

In [42]:
len(np.arange(agent.action_space[0].n))//2

10

Define RL agent

In [52]:
def eval_furthest_action_attack(agent, env, ART_atk, time_steps:int=None, mask:list=None):
    """Evaluates an SB3 agent subject to untargeted observation perturbations generated by an ART evasion attack,
    starting p[oints are added for the bb attack, which requires initialization from a sample of the target class"""

    obs_list = []
    adv_obs_list = []
    adv_a_list = []
    action_list = []
    asr = tasr = 0
    kwargs = dict()

    action_space = np.arange(agent.action_space[0].n) #array of discrete actions, assumes 1d
    action_mid = len(np.arange(agent.action_space[0].n))//2

    observations = env.reset()
    if time_steps is None:
        time_steps = env.time_steps - 1
    if mask is not None:
        kwargs['mask'] = mask

    pbar = tqdm(total=time_steps)
    for step in range(time_steps):

        obs_list.append(observations)
        a, _ = agent.predict(observations, deterministic=True)
        action_list.append(a)

        #off by one error here,
        if a[0] < action_mid: #try targets that flip the (dis)charge of the action
            max = action_space[-1] #add 1?
            min = action_space[action_mid]
        else:
            max = action_space[action_mid - 1]
            min = action_space[0] #add 1?

#instead of a binary search, a batch of idendical smaples paired with different targets could be computed
    #then the batch is test on the victim and the successful sample chosen.
            
        kwargs['targeted'] = True
        target = min
        kwargs['y'] = to_categorical([target], nb_classes=agent.action_space[0].n) #one-hot encode int
        adv_obs = np.squeeze(ART_atk.generate(np.expand_dims(observations, axis=0), **kwargs))
        adv_a, _ = agent.predict(adv_obs, deterministic=True)

        if target==adv_a[0]: #check if the action matches the intended target
            asr+=1
            tasr+=1
            best_adv_obs = adv_obs
            min += 1 #don't test this twice
            while(max > min): #binary search for furthest action
                target = min + (max - min)//2
                kwargs['y'] = to_categorical([target], nb_classes=agent.action_space[0].n)
                adv_obs = np.squeeze(ART_atk.generate(np.expand_dims(observations, axis=0), **kwargs))
                adv_a, _ = agent.predict(adv_obs, deterministic=True)
                if target==adv_a[0]: #check if the action matches the intended target
                    best_adv_obs = adv_obs
                    min = target
                else:
                    max = target
            adv_obs_list.append(best_adv_obs)
        else: #closest target failed
            kwargs['targeted'] = False #use untargeted attack
            kwargs['y'] = None
            adv_obs = np.squeeze(ART_atk.generate(np.expand_dims(observations, axis=0), **kwargs))
            adv_a, _ = agent.predict(adv_obs, deterministic=True)
            if adv_a[0]!=a[0]: #check if untargeted attack succeeded
                asr+=1
                adv_obs_list.append(adv_obs)
            else:
                adv_obs_list.append(np.array([np.nan]*agent.observation_space.shape[0])) #same shape as observations
            
        
        adv_a_list.append(adv_a)

        observations, _, _, _ = env.step(adv_a)

        #update progress bar including asr
        pbar.update(1)
        pbar.set_postfix({'ASR': asr/(step + 1),'Targeted ASR': tasr/(step + 1)}, refresh=True)
        if env.done:
            break
    
    pbar.close()
    asr/=time_steps
    return utils.format_kpis(env), np.array(obs_list), np.array(adv_obs_list), np.array(action_list), np.array(adv_a_list), asr

In [44]:
init = 50
iter = int(500/init)
params = dict(
    loss_type='difference_logits_ratio', 
    eps = EPS,
    eps_step = 2*EPS,
    batch_size=1,
    nb_random_init=init, #5, lower values speed crafting
    max_iter=iter, #iterations per restart
    norm='inf', #->l2 ->l1 most restrictive 
    verbose=False,
)
attack = utils.define_attack(agent, ACG, ART_kwargs=params)

In [53]:
%%time
kpis, obs, adv_obs, actions, adv_actions, asr = eval_furthest_action_attack(agent, 
                                                                            env, 
                                                                            attack,
                                                                            time_steps=None,
                                                                            mask=observation_masks)

100%|██████████| 8759/8759 [11:05<00:00, 13.16it/s, ASR=0.997, Targeted ASR=0]  


CPU times: total: 1min 50s
Wall time: 11min 6s


For eps up to 0.1, the targeted attack never succeeds

In [46]:
display(kpis)

cost_function
annual_peak_average                      1.289772
carbon_emissions_total                   0.936198
cost_total                               0.865539
daily_one_minus_load_factor_average      1.119091
daily_peak_average                       1.053599
electricity_consumption_total            0.941203
monthly_one_minus_load_factor_average    1.004105
ramping_average                          1.444045
zero_net_energy                          1.112498
Name: District, dtype: float64

In [47]:
kpi = kpis

In [48]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print(f'{kpi_savename} updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print(f'{kpi_savename} created')

20 bin PPO 500 results\furthest action results/KPIs.csv updated


In [49]:
df_obs = pd.DataFrame(obs)
df_obs.columns = cols
df_obs['a'] = actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+'a-obs.csv')

In [50]:
df_obs = pd.DataFrame(adv_obs)
df_obs.columns = cols
df_obs['a'] = adv_actions
df_obs.to_csv(SAVE_DIR+ATK_NAME+' adv a-obs.csv')

In [51]:
asr_savename = SAVE_DIR+'ASRs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = asr
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([asr])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

20 bin PPO 500 results\furthest action results/ASRs.csv updated
